In [1]:
# 데이터 붙이기 => 번호 컬럼만 따로 가져오기 => 기존 데이터에 내용 추가해보기

In [4]:
import numpy as np
import pandas as pd
import requests 
import time
from bs4 import BeautifulSoup 
from glob import glob


# 21.09.20 기준 https://kr.investing.com/stock-screener/ 총 54페이분량 합치기

file_names = glob("/Users/ppangppang/Desktop/ssac/finance_data/*.csv")
file_names

total = pd.DataFrame()
for file_name in file_names:
    temp = pd.read_csv(file_name)
    total = pd.concat([total, temp])

# 데이터를 편하게 보기위해 옵션 설정
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

In [6]:
# 사용할 데이터만가져오기, 기존 데이터에 있는 종가는 현재 가격이 아님
total.index = range(len(total))
df = total[['종목','기호','거래량']]
df

,종목,기호,거래량
0,동아화성,41930,0.00
1,SBS미디어홀딩스,101060,186.45K
2,제노포커스,187420,0.00
3,씨티씨바이오,60590,0.00
4,Amosense Co,357580,0.00
...,...,...,...
2653,애니플러스,310200,0.00
2654,삼륭물산,14970,0.00
2655,제일바이오,52670,0.00
2656,한국가구,4590,0.00


In [7]:
# 종목만 따로 떼오기. str로 변환안하면 0을 미포함하는 경우가 생김
temp = df['기호'].values
temp2 = list(map(str, temp))

for num in range(len(temp2)-1):
    if len(temp2[num]) ==5:
        temp2[num] = '0' + temp2[num]
    elif len(temp2[num]) ==4:
        temp2[num] = '00' + temp2[num]
    elif len(temp2[num]) ==3:
        temp2[num] = '000' + temp2[num]
    elif len(temp2[num]) ==2:
        temp2[num] = '0000' + temp2[num]
    elif len(temp2[num]) ==1:
        temp2[num] = '00000' + temp2[num]
temp = temp2

In [8]:
# str형으로 변경됐는지 최종확인
for num in range(len(temp)-1):
    if len(temp[num]) !=6:
        print(num)  # 	미래에셋맵스프런티어브라질 8자리로 되어있는 펀드같음 신경 안써도 되느낮료

2455


In [9]:
# 종목코드 변환한것 df에 넣기
temp=np.array(temp)
temp = np.ravel(temp, order='C')
temp = pd.DataFrame({"기호" : temp})
# temp
df1 = df[['종목']]
df2 = df1.join(temp, how='outer')
df3 = df2.join(df[['거래량']], how='outer')
df_set = df3

In [11]:
df_set.tail()

,종목,기호,거래량
2653,애니플러스,310200,0.00
2654,삼륭물산,014970,0.00
2655,제일바이오,052670,0.00
2656,한국가구,004590,0.00
2657,동양파일,228340,0.00


In [12]:
code= df_set['기호']
len(code)

2658

In [445]:
# 종목번호를 잘못입력해도 연결되고, html 변환도 오류 안뜬다
# url = f'https://finance.naver.com/item/main.nhn?code=000000'
# response = requests.get(url)
# response
# html = BeautifulSoup(response.text, 'html.parser')

In [446]:
# 오류 뜨기전 사용하던 코드
# price = []

# for num in code:
#     url = f'https://finance.naver.com/item/main.nhn?code={num}'

#     print(f'{num}종목 현재가격 크롤링중 입니다.')
#     seed = np.random.randint(100) # seed 설정
#     np.random.seed(seed)    # 시드 생성
#     a = np.random.randint(5)  # 시드에서 난수 생성
#     time.sleep(a)     # 코드실행 지연시키는 코드
    
#     response = requests.get(url)

#     if(response.status_code == requests.codes.ok):
#         html = BeautifulSoup(response.text, 'html.parser')

#     else:
# #         print(f'{num}종목 크롤링 실패, 값을 다시 확인해주세요(금일종가 0원으로 처리)')
#         print(f"크롤링 실패. 종목번호 : {num} 을 확인해 주세요")
# #         price.append('0')
# #         continue

#     price.append(html.select('p.no_today')[0].text)


In [19]:
# 현재가격 업로드(210922 기준 최종)
price = []

for num in code:
    url = f'https://finance.naver.com/item/main.nhn?code={num}'

    print(f'{num}종목 현재가격 크롤링중 입니다.')
    seed = np.random.randint(100) # seed 설정
    np.random.seed(seed)    # 시드 생성
    a = np.random.randint(5)  # 시드에서 난수 생성
    time.sleep(a)     # 코드실행 지연시키는 코드
    
    response = requests.get(url)

    if(response.status_code == requests.codes.ok):
        html = BeautifulSoup(response.text, 'html.parser')
           
    else:
        print(f"크롤링 실패. 종목번호 : {num} 을 확인해 주세요")

    try:
        price.append(html.select('p.no_today')[0].text)

    except:
        print(f'{num}종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)')
        price.append('0')

print("크롤링 완료")

041930종목 현재가격 크롤링중 입니다.
101060종목 현재가격 크롤링중 입니다.
187420종목 현재가격 크롤링중 입니다.
060590종목 현재가격 크롤링중 입니다.
357580종목 현재가격 크롤링중 입니다.
092230종목 현재가격 크롤링중 입니다.
290670종목 현재가격 크롤링중 입니다.
000430종목 현재가격 크롤링중 입니다.
007160종목 현재가격 크롤링중 입니다.
234080종목 현재가격 크롤링중 입니다.
083420종목 현재가격 크롤링중 입니다.
060560종목 현재가격 크롤링중 입니다.
306040종목 현재가격 크롤링중 입니다.
004960종목 현재가격 크롤링중 입니다.
143160종목 현재가격 크롤링중 입니다.
011500종목 현재가격 크롤링중 입니다.
038070종목 현재가격 크롤링중 입니다.
270870종목 현재가격 크롤링중 입니다.
092220종목 현재가격 크롤링중 입니다.
082800종목 현재가격 크롤링중 입니다.
083310종목 현재가격 크롤링중 입니다.
005680종목 현재가격 크롤링중 입니다.
021320종목 현재가격 크롤링중 입니다.
211270종목 현재가격 크롤링중 입니다.
083450종목 현재가격 크롤링중 입니다.
253840종목 현재가격 크롤링중 입니다.
086980종목 현재가격 크롤링중 입니다.
004380종목 현재가격 크롤링중 입니다.
074430종목 현재가격 크롤링중 입니다.
012700종목 현재가격 크롤링중 입니다.
354200종목 현재가격 크롤링중 입니다.
023450종목 현재가격 크롤링중 입니다.
004890종목 현재가격 크롤링중 입니다.
256940종목 현재가격 크롤링중 입니다.
238090종목 현재가격 크롤링중 입니다.
002210종목 현재가격 크롤링중 입니다.
203650종목 현재가격 크롤링중 입니다.
006620종목 현재가격 크롤링중 입니다.
288330종목 현재가격 크롤링중 입니다.
036890종목 현재가격 크롤링중 입니다.
039240종목 현재가격 크롤링중 입니다.
053290종목 현재가격 크롤

115440종목 현재가격 크롤링중 입니다.
079810종목 현재가격 크롤링중 입니다.
021820종목 현재가격 크롤링중 입니다.
076610종목 현재가격 크롤링중 입니다.
024120종목 현재가격 크롤링중 입니다.
031510종목 현재가격 크롤링중 입니다.
019770종목 현재가격 크롤링중 입니다.
005860종목 현재가격 크롤링중 입니다.
036180종목 현재가격 크롤링중 입니다.
001810종목 현재가격 크롤링중 입니다.
088910종목 현재가격 크롤링중 입니다.
006980종목 현재가격 크롤링중 입니다.
150900종목 현재가격 크롤링중 입니다.
215100종목 현재가격 크롤링중 입니다.
376980종목 현재가격 크롤링중 입니다.
047310종목 현재가격 크롤링중 입니다.
203400종목 현재가격 크롤링중 입니다.
203400종목 현재가격 크롤링중 입니다.
142760종목 현재가격 크롤링중 입니다.
285490종목 현재가격 크롤링중 입니다.
251970종목 현재가격 크롤링중 입니다.
298060종목 현재가격 크롤링중 입니다.
138610종목 현재가격 크롤링중 입니다.
033920종목 현재가격 크롤링중 입니다.
115960종목 현재가격 크롤링중 입니다.
206560종목 현재가격 크롤링중 입니다.
025770종목 현재가격 크롤링중 입니다.
033240종목 현재가격 크롤링중 입니다.
241840종목 현재가격 크롤링중 입니다.
100130종목 현재가격 크롤링중 입니다.
033500종목 현재가격 크롤링중 입니다.
042370종목 현재가격 크롤링중 입니다.
001275종목 현재가격 크롤링중 입니다.
078590종목 현재가격 크롤링중 입니다.
043610종목 현재가격 크롤링중 입니다.
244460종목 현재가격 크롤링중 입니다.
376190종목 현재가격 크롤링중 입니다.
900140종목 현재가격 크롤링중 입니다.
005500종목 현재가격 크롤링중 입니다.
044340종목 현재가격 크롤링중 입니다.
001460종목 현재가격 크롤링중 입니다.
122990종목 현재가격 크롤

386580종목 현재가격 크롤링중 입니다.
101680종목 현재가격 크롤링중 입니다.
189350종목 현재가격 크롤링중 입니다.
321260종목 현재가격 크롤링중 입니다.
368770종목 현재가격 크롤링중 입니다.
355150종목 현재가격 크롤링중 입니다.
351340종목 현재가격 크롤링중 입니다.
241510종목 현재가격 크롤링중 입니다.
241510종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
224020종목 현재가격 크롤링중 입니다.
333050종목 현재가격 크롤링중 입니다.
317030종목 현재가격 크롤링중 입니다.
086460종목 현재가격 크롤링중 입니다.
208890종목 현재가격 크롤링중 입니다.
340120종목 현재가격 크롤링중 입니다.
353490종목 현재가격 크롤링중 입니다.
348840종목 현재가격 크롤링중 입니다.
353060종목 현재가격 크롤링중 입니다.
365590종목 현재가격 크롤링중 입니다.
058220종목 현재가격 크롤링중 입니다.
083470종목 현재가격 크롤링중 입니다.
313750종목 현재가격 크롤링중 입니다.
307070종목 현재가격 크롤링중 입니다.
307070종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
367360종목 현재가격 크롤링중 입니다.
291210종목 현재가격 크롤링중 입니다.
291210종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
341160종목 현재가격 크롤링중 입니다.
262830종목 현재가격 크롤링중 입니다.
262830종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
310870종목 현재가격 크롤링중 입니다.
271850종목 현재가격 크롤링중 입니다.
366330종목 현재가격 크롤링중 입니다.
183300종목 현재가격 크롤링중 입니다.
069080종목 현재가격 크롤링중 입니다.
033640종목 현재가격 크롤링중 입니다.
003280종목 현재가격 크롤링중 입니다.
006040종목 현재가격 크롤링중 입니다.
082270종목

025540종목 현재가격 크롤링중 입니다.
057050종목 현재가격 크롤링중 입니다.
380320종목 현재가격 크롤링중 입니다.
161570종목 현재가격 크롤링중 입니다.
278990종목 현재가격 크롤링중 입니다.
243870종목 현재가격 크롤링중 입니다.
114120종목 현재가격 크롤링중 입니다.
279060종목 현재가격 크롤링중 입니다.
135160종목 현재가격 크롤링중 입니다.
253610종목 현재가격 크롤링중 입니다.
344860종목 현재가격 크롤링중 입니다.
053060종목 현재가격 크롤링중 입니다.
222670종목 현재가격 크롤링중 입니다.
064510종목 현재가격 크롤링중 입니다.
250030종목 현재가격 크롤링중 입니다.
058450종목 현재가격 크롤링중 입니다.
053660종목 현재가격 크롤링중 입니다.
238170종목 현재가격 크롤링중 입니다.
058530종목 현재가격 크롤링중 입니다.
224810종목 현재가격 크롤링중 입니다.
045890종목 현재가격 크롤링중 입니다.
208850종목 현재가격 크롤링중 입니다.
039230종목 현재가격 크롤링중 입니다.
221610종목 현재가격 크롤링중 입니다.
010580종목 현재가격 크롤링중 입니다.
053110종목 현재가격 크롤링중 입니다.
309900종목 현재가격 크롤링중 입니다.
317320종목 현재가격 크롤링중 입니다.
331660종목 현재가격 크롤링중 입니다.
236030종목 현재가격 크롤링중 입니다.
363260종목 현재가격 크롤링중 입니다.
141020종목 현재가격 크롤링중 입니다.
178600종목 현재가격 크롤링중 입니다.
103230종목 현재가격 크롤링중 입니다.
257990종목 현재가격 크롤링중 입니다.
331380종목 현재가격 크롤링중 입니다.
149010종목 현재가격 크롤링중 입니다.
215570종목 현재가격 크롤링중 입니다.
299670종목 현재가격 크롤링중 입니다.
038340종목 현재가격 크롤링중 입니다.
369370종목 현재가격 크롤링중 입니다.
354230종목 현재가격 크롤

065940종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
197210종목 현재가격 크롤링중 입니다.
197210종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
225850종목 현재가격 크롤링중 입니다.
017680종목 현재가격 크롤링중 입니다.
017680종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
225860종목 현재가격 크롤링중 입니다.
204990종목 현재가격 크롤링중 입니다.
204990종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
094190종목 현재가격 크롤링중 입니다.
094190종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
030270종목 현재가격 크롤링중 입니다.
030270종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
054340종목 현재가격 크롤링중 입니다.
054340종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
028040종목 현재가격 크롤링중 입니다.
028040종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
080440종목 현재가격 크롤링중 입니다.
080440종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
189540종목 현재가격 크롤링중 입니다.
189540종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
329050종목 현재가격 크롤링중 입니다.
329050종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
220110종목 현재가격 크롤링중 입니다.
220110종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
141080종목 현재가격 크롤링중 입니다.
004490종목 현재가격 크롤링중 입니다.
060250종목 현재가격 크롤링중 입니다.
039130종목 현재가격 크롤링중 입니다.
214320종목 현재가격 크롤링중 입니다.
079160종목 현재가격 크롤링중 입니다.
009420종목 현재가격 크롤

066830종목 현재가격 크롤링중 입니다.
127160종목 현재가격 크롤링중 입니다.
115610종목 현재가격 크롤링중 입니다.
064090종목 현재가격 크롤링중 입니다.
065420종목 현재가격 크롤링중 입니다.
056090종목 현재가격 크롤링중 입니다.
162120종목 현재가격 크롤링중 입니다.
200580종목 현재가격 크롤링중 입니다.
011230종목 현재가격 크롤링중 입니다.
155900종목 현재가격 크롤링중 입니다.
160600종목 현재가격 크롤링중 입니다.
214310종목 현재가격 크롤링중 입니다.
044780종목 현재가격 크롤링중 입니다.
227610종목 현재가격 크롤링중 입니다.
005030종목 현재가격 크롤링중 입니다.
113810종목 현재가격 크롤링중 입니다.
003680종목 현재가격 크롤링중 입니다.
029480종목 현재가격 크롤링중 입니다.
900270종목 현재가격 크롤링중 입니다.
219550종목 현재가격 크롤링중 입니다.
065690종목 현재가격 크롤링중 입니다.
020400종목 현재가격 크롤링중 입니다.
112190종목 현재가격 크롤링중 입니다.
071950종목 현재가격 크롤링중 입니다.
134060종목 현재가격 크롤링중 입니다.
054410종목 현재가격 크롤링중 입니다.
090740종목 현재가격 크롤링중 입니다.
015260종목 현재가격 크롤링중 입니다.
002680종목 현재가격 크롤링중 입니다.
001420종목 현재가격 크롤링중 입니다.
131100종목 현재가격 크롤링중 입니다.
010640종목 현재가격 크롤링중 입니다.
101400종목 현재가격 크롤링중 입니다.
004920종목 현재가격 크롤링중 입니다.
110020종목 현재가격 크롤링중 입니다.
323230종목 현재가격 크롤링중 입니다.
073190종목 현재가격 크롤링중 입니다.
026910종목 현재가격 크롤링중 입니다.
099520종목 현재가격 크롤링중 입니다.
210610종목 현재가격 크롤링중 입니다.
210610종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 

083370종목 현재가격 크롤링중 입니다.
083370종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
083380종목 현재가격 크롤링중 입니다.
083380종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
373340종목 현재가격 크롤링중 입니다.
349720종목 현재가격 크롤링중 입니다.
058420종목 현재가격 크롤링중 입니다.
323280종목 현재가격 크롤링중 입니다.
276920종목 현재가격 크롤링중 입니다.
276920종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
232330종목 현재가격 크롤링중 입니다.
232330종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
311270종목 현재가격 크롤링중 입니다.
311270종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
285770종목 현재가격 크롤링중 입니다.
257730종목 현재가격 크롤링중 입니다.
257730종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
279410종목 현재가격 크롤링중 입니다.
279410종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
121890종목 현재가격 크롤링중 입니다.
114570종목 현재가격 크롤링중 입니다.
335870종목 현재가격 크롤링중 입니다.
310840종목 현재가격 크롤링중 입니다.
310840종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
323210종목 현재가격 크롤링중 입니다.
347140종목 현재가격 크롤링중 입니다.
179720종목 현재가격 크롤링중 입니다.
242350종목 현재가격 크롤링중 입니다.
217910종목 현재가격 크롤링중 입니다.
270020종목 현재가격 크롤링중 입니다.
140290종목 현재가격 크롤링중 입니다.
264290종목 현재가격 크롤링중 입니다.
264290종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
238500종목 현재가격 크롤링중 입니다.
2670

003670종목 현재가격 크롤링중 입니다.
036570종목 현재가격 크롤링중 입니다.
090430종목 현재가격 크롤링중 입니다.
010950종목 현재가격 크롤링중 입니다.
006920종목 현재가격 크롤링중 입니다.
080010종목 현재가격 크롤링중 입니다.
017370종목 현재가격 크롤링중 입니다.
013000종목 현재가격 크롤링중 입니다.
267790종목 현재가격 크롤링중 입니다.
044480종목 현재가격 크롤링중 입니다.
218150종목 현재가격 크롤링중 입니다.
018620종목 현재가격 크롤링중 입니다.
196700종목 현재가격 크롤링중 입니다.
225650종목 현재가격 크롤링중 입니다.
225650종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
294630종목 현재가격 크롤링중 입니다.
049120종목 현재가격 크롤링중 입니다.
149980종목 현재가격 크롤링중 입니다.
048470종목 현재가격 크롤링중 입니다.
004105종목 현재가격 크롤링중 입니다.
284420종목 현재가격 크롤링중 입니다.
284420종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
225430종목 현재가격 크롤링중 입니다.
213090종목 현재가격 크롤링중 입니다.
094860종목 현재가격 크롤링중 입니다.
069110종목 현재가격 크롤링중 입니다.
051780종목 현재가격 크롤링중 입니다.
012620종목 현재가격 크롤링중 입니다.
101240종목 현재가격 크롤링중 입니다.
187220종목 현재가격 크롤링중 입니다.
049520종목 현재가격 크롤링중 입니다.
044380종목 현재가격 크롤링중 입니다.
127120종목 현재가격 크롤링중 입니다.
067920종목 현재가격 크롤링중 입니다.
054930종목 현재가격 크롤링중 입니다.
031860종목 현재가격 크롤링중 입니다.
037760종목 현재가격 크롤링중 입니다.
002820종목 현재가격 크롤링중 입니다.
001080종목 현재가격 크롤링중 입니다.
279600종목 현재가격 크롤링중 입니다.


001270종목 현재가격 크롤링중 입니다.
061250종목 현재가격 크롤링중 입니다.
304840종목 현재가격 크롤링중 입니다.
094850종목 현재가격 크롤링중 입니다.
101140종목 현재가격 크롤링중 입니다.
214610종목 현재가격 크롤링중 입니다.
009470종목 현재가격 크롤링중 입니다.
011760종목 현재가격 크롤링중 입니다.
039860종목 현재가격 크롤링중 입니다.
003465종목 현재가격 크롤링중 입니다.
003160종목 현재가격 크롤링중 입니다.
096240종목 현재가격 크롤링중 입니다.
043370종목 현재가격 크롤링중 입니다.
300120종목 현재가격 크롤링중 입니다.
033830종목 현재가격 크롤링중 입니다.
095190종목 현재가격 크롤링중 입니다.
002710종목 현재가격 크롤링중 입니다.
317690종목 현재가격 크롤링중 입니다.
229500종목 현재가격 크롤링중 입니다.
144620종목 현재가격 크롤링중 입니다.
207760종목 현재가격 크롤링중 입니다.
950200종목 현재가격 크롤링중 입니다.
018120종목 현재가격 크롤링중 입니다.
049950종목 현재가격 크롤링중 입니다.
125210종목 현재가격 크롤링중 입니다.
348350종목 현재가격 크롤링중 입니다.
002100종목 현재가격 크롤링중 입니다.
215090종목 현재가격 크롤링중 입니다.
089890종목 현재가격 크롤링중 입니다.
103590종목 현재가격 크롤링중 입니다.
277880종목 현재가격 크롤링중 입니다.
271400종목 현재가격 크롤링중 입니다.
271400종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)
082850종목 현재가격 크롤링중 입니다.
004700종목 현재가격 크롤링중 입니다.
006220종목 현재가격 크롤링중 입니다.
217500종목 현재가격 크롤링중 입니다.
290380종목 현재가격 크롤링중 입니다.
003720종목 현재가격 크롤링중 입니다.
046940종목 현재가격 크롤링중 입니다.
238490종목 현재가격 크롤링중 입

In [20]:
# price 정리

code_price =[]

for num in price:
    temp = []
        # 현재가격 크롤링 되지 않는 데이터를 html.select형태로 변환해서 작업해버리자
    if num == '0':
        num = '\n\n0\n0\n\n'
        temp.append(num.split('\n'))
        temp=np.array(temp)
        code_price.append(temp[0,2:3])

    else:
        temp.append(num.split('\n'))
        temp=np.array(temp)
        code_price.append(temp[0,2:3])

code_price = np.ravel(code_price, order='C')
code_price = pd.DataFrame({"현재가격" : code_price})

In [22]:
code_price.tail()

,현재가격
2653,"3,760"
2654,"6,990"
2655,"3,680"
2656,"6,600"
2657,"5,140"


In [31]:
# 현재가격 업데이트
df_daily = df_set.join(code_price, how='outer')

In [36]:
# 최종확인
df_daily.tail()

,종목,기호,거래량,현재가격
2653,애니플러스,310200,0.00,"3,760"
2654,삼륭물산,014970,0.00,"6,990"
2655,제일바이오,052670,0.00,"3,680"
2656,한국가구,004590,0.00,"6,600"
2657,동양파일,228340,0.00,"5,140"


In [39]:
df_daily.to_csv('210924_finance_data.csv',index = False)

In [40]:
# 저장된 파일 상태 확인
test = pd.read_csv('210924_finance_data.csv')
test.tail()

,종목,기호,거래량,현재가격
2653,애니플러스,310200,0.00,"3,760"
2654,삼륭물산,014970,0.00,"6,990"
2655,제일바이오,052670,0.00,"3,680"
2656,한국가구,004590,0.00,"6,600"
2657,동양파일,228340,0.00,"5,140"
